## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium

## Read-in

In [2]:
df = pd.read_csv('LISF_February_and_March.csv',dtype='str')

## Clean, drop, and convert

In [3]:
df = df.drop(columns=['Unnamed: 0.2','Unnamed: 0.1','Unnamed: 0'])

In [4]:
df = df.drop_duplicates()

In [5]:
df['Doc Recorded'] = pd.to_datetime(df['Doc Recorded'])

## Data Stuff

In [6]:
# Count number of February dates
feb_dates_count = len(df.loc[df['Doc Recorded'].dt.month == 2])

print(f'Number of February dates: {feb_dates_count}')

Number of February dates: 807


In [7]:
# Count number of February dates
mar_dates_count = len(df.loc[df['Doc Recorded'].dt.month == 3])

print(f'Number of March dates: {mar_dates_count}')

Number of March dates: 1010


In [8]:
# split the '1st PIN' column into two columns
df[['folio_number', 'address']] = df['1st PIN'].str.split(' ', 1, expand=True)
# Add Illinois
df['geo_address'] = df['address'] + ' Illinois'

In [9]:
df['1st Grantor'] = df['1st Grantor'].fillna('NA')
df['1st Grantee'] = df['1st Grantee'].fillna('NA')

In [10]:
municipal_authority = ['city', 'town', 'municipality', 'village']  # list of municipal authority keywords
df = df[~df['1st Grantor'].str.contains('|'.join(municipal_authority), case=False) & 
        ~df['1st Grantee'].str.contains('|'.join(municipal_authority), case=False)]

In [11]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [12]:
df = df.dropna(subset='address')

In [13]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [14]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

In [25]:
# create new column and assign colors based on month
df['COLOR'] = df['Doc Recorded'].apply(lambda x: 'pink' if x.month == 2 else 'red' if x.month == 3 else '')

In [26]:
df['COLOR'].value_counts()

red     778
pink    662
Name: COLOR, dtype: int64

## HTML Popup Formatter

In [16]:
df.columns

Index(['View Doc', 'Doc Number', 'Doc Recorded', 'Doc Executed', 'Doc Type',
       'Consi. Amt.', '1st Grantor', '1st Grantee', 'Assoc. Doc#', '1st PIN',
       'deed_urls', 'folio_number', 'address', 'geo_address', 'geocoded',
       'lat', 'lon'],
      dtype='object')

In [33]:
def popup_html(row):
    grantor = row['1st Grantor']
    grantee = row['1st Grantee']
    folio = row['folio_number']
    address = row['address']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Lendor: </strong>{}'''.format(grantor) + '''<br>
    <strong>Lendee: </strong>{}'''.format(grantee) + '''<br>
    <strong>PIN: </strong>{}'''.format(folio) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    </html>
    '''
    return html

In [36]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Cook County Foreclosures')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('February & March')


### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_pink = folium.FeatureGroup(name='February')
fg_red = folium.FeatureGroup(name='March')

for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'pink':
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_pink)
    else:
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_red)

# Add the FeatureGroups to the map
fg_pink.add_to(m)
fg_red.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map

In [37]:
m.save('index.html')

## Map URL Snagger

In [2]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/lis_pendens_scraper


## Stats for story

In [42]:
# Count number of February dates
feb_dates_count = len(df.loc[df['Doc Recorded'].dt.month == 2])

print(f'Number of February dates: {feb_dates_count}')

# Count number of February dates
mar_dates_count = len(df.loc[df['Doc Recorded'].dt.month == 3])

print(f'Number of March dates: {mar_dates_count}')

Number of February dates: 662
Number of March dates: 778


In [43]:
feb = df.loc[df['Doc Recorded'].dt.month == 2]
mar = df.loc[df['Doc Recorded'].dt.month == 3]

In [51]:
feb['1st Grantor'].value_counts().head(60)

NATIONSTAR MTG LLC                        40
US BK NATL ASSN                           39
US BK NATL ASSN TR                        37
LAKEVIEW LOAN SERVICING LLC               35
NEWREZ LLC                                32
US BK TRUST NATL ASSN TR                  28
WILMINGTON SAV FUND SOC FSB TR            26
WELLS FARGO BK NA                         21
FREEDOM MTG CORP                          18
PENNYMAC LOAN SERVICES LLC                17
CARRINGTON MTG SERVICES LLC               16
MIDFIRST BK                               14
DEUTSCHE BK NATL TRUST CO TR              13
LOANDEPOTCOM LLC                          11
JPMORGAN CHASE BK NATL ASSN               10
PNC BK NATL ASSN                           9
THE BK OF NEW YORK MELLON TR               9
FEDERAL HOME LOAN MTG CORP TR              7
ROCKET MTG LLC                             7
FIFTH THIRD BK NATL ASSN                   7
UNITED WHOLESALE MTG LLC                   6
CITIZENS BK NA                             5
SELECT POR

In [50]:
feb['1st Grantee'].value_counts()

UNKNOWN                                               8
CHICAGO TITLE LAND TRUST CO TR                        4
HAB GRP LLC                                           4
THE MRW ANNUITY TRUST I                               2
ROJAS MARICELA HERNANDEZ                              2
                                                     ..
MAEBANE DAVID W                                       1
MARTINEZ DARIO                                        1
UNKNOWN HEIRS OF CARL JOHNSON                         1
CARBAJAL MAYRA                                        1
6606-08 NORTH SEELEY BUILDING CONDOMINIUM ASSOCIAT    1
Name: 1st Grantee, Length: 638, dtype: int64

In [47]:
mar['1st Grantor'].value_counts()

US BK NATL ASSN                    56
LAKEVIEW LOAN SERVICING LLC        46
US BK NATL ASSN TR                 42
NATIONSTAR MTG LLC                 39
US BK TRUST NATL ASSN TR           36
                                   ..
ATAR NASRIN                         1
SANTANDER BK NA                     1
MAO SONGAN                          1
ANTLER MTG LOAN TRUST 2021-RTL1     1
THE BK OF NEW YORK MELLOM TR        1
Name: 1st Grantor, Length: 196, dtype: int64

In [52]:
mar['1st Grantee'].value_counts()

CHICAGO TITLE LAND TRUST CO TR    7
UNKNOWN                           4
DELEE HOLDINGS LLC                3
SMITH JAMES                       3
BONILLA JOSE FABIAN               2
                                 ..
STEINFINK DEBRA B                 1
RIOS GUIDO J                      1
LYTLE DOUGLAS GDN                 1
RICHARDSON GERALD                 1
NAVARRO RENATO J                  1
Name: 1st Grantee, Length: 759, dtype: int64

In [54]:
df.to_csv('feb_mar_foreclosures.csv')